In [1]:
suppressMessages(here::i_am('R_analysis/notebooks/SI_coefficient_table.ipynb'))
CODE = here::here()
source(glue::glue("{CODE}/R_analysis/init.R"))

Loading required package: pacman



In [7]:
# Load functions
local_load('analysis/regress.R')
local_load('analysis/clean_data.R')
# Load arguments
local_load('analysis/main_analysis_args.R')

In [28]:
crop_name = 'corn'
t_list = list()
for (source in c('prism', 'era5', 'gmfd')) {
    for (crop in c('corn', 'soy')) {
        
        KDD_lower = KDD_lower_list[[crop]][[source]]
        df = load_panel(
            source, crop,
            year_min, year_max,
            growing_season_min, growing_season_max,
            GDD_lower, KDD_lower,
            temp_lower, temp_upper
        )

        tdf = df |> mutate(
            GDD = GDD / 1e+03,
            KDD = KDD / 1e+03,
            prec = prec / 1e+03,
            prec2 = prec2 / 1e+06
        )

        m1 = run_yield_regression(tdf, spec='piecewise', cluster='fips + state')

        m2 = run_yield_regression(tdf, spec='piecewise', cluster='year')

        m3 = run_yield_regression(tdf, spec='piecewise', cluster=conley(1000))

        m4 = run_yield_regression(tdf, spec='piecewise', controls='fips + year', cluster='fips + state')

        m5 = run_yield_regression(tdf, spec='piecewise', controls='fips + year', cluster='year')

        m6 = run_yield_regression(tdf, spec='piecewise', controls='fips + year', cluster=conley(1000))

        t_list[[crop]][[source]] = etable(m1, m2, m3, m4, m5, m6, tex=TRUE)
    }
}

In [29]:
t_list

$corn
$corn$prism
\begingroup
\centering
\begin{tabular}{lcccccc}
   \tabularnewline \midrule \midrule
   Dependent Variable: & \multicolumn{6}{c}{log\_yield}\\
   Model:       & (1)             & (2)             & (3)             & (4)             & (5)             & (6)\\  
   \midrule
   \emph{Variables}\\
   GDD          & 0.2865$^{***}$  & 0.2865$^{***}$  & 0.2865$^{***}$  & 0.2630$^{***}$  & 0.2630$^{**}$   & 0.2630$^{***}$\\   
                & (0.0640)        & (0.0908)        & (0.1090)        & (0.0664)        & (0.1097)        & (0.0943)\\   
   KDD          & -5.926$^{***}$  & -5.926$^{***}$  & -5.926$^{***}$  & -5.655$^{***}$  & -5.655$^{***}$  & -5.655$^{***}$\\   
                & (0.5910)        & (0.3924)        & (0.8827)        & (0.5902)        & (0.5196)        & (0.9501)\\   
   prec         & 0.8643$^{***}$  & 0.8643$^{***}$  & 0.8643$^{***}$  & 0.7256$^{***}$  & 0.7256$^{***}$  & 0.7256$^{***}$\\   
                & (0.1665)        & (0.1736)        & (0.0827